In [263]:
import numpy as np
import pandas as pd
import random

# GA Class

In [264]:
class MancalaGA:
    
    def __init__(self, self_state, opponent_state, gen, popsize, generation, pc, pm):
        self.num_of_gen = gen
        self.pop_size = popsize
        self.num_of_generation = generation
        self.pc = pc
        self.pm = pm
        self.self_state = self_state # list state ai (lubang) *default [7,7,7,7,7,7,7]
        self.opponent_state = opponent_state # list state player (lubang) *default [7,7,7,7,7,7,7]
        
    
    def individu(self):
        return np.array([np.random.uniform(0,100) for i in range(self.num_of_gen)])
    
    def population(self):
        return np.array([self.individu() for i in range(self.pop_size)])
    
    def debug_lubang_representation(self):
        global h1,h2,h3,h4,h5
        populasi = self.population()
        
        
        for idx, lubang in enumerate(self.self_state):
            idx_lubang = idx
            isi_lubang = lubang

            # get value tiap heuristik
            h1 = self.h1(idx_lubang, isi_lubang)
            h2 = self.h2(idx_lubang, isi_lubang)
            h3 = self.h3(idx_lubang, isi_lubang)
            h4 = self.h4(idx_lubang, isi_lubang)
            h5 = self.h5(idx_lubang, isi_lubang)


            print(f"ini lubang ke {idx_lubang} dengan isi isi_lubang {isi_lubang}")
            print(f"nilai h1 adalah {h1}")
            print(f"nilai h2 adalah {h2}")
            print(f"nilai h3 adalah {h3}")
            print(f"nilai h4 adalah {h4}")
            print(f"nilai h5 adalah {h5}")
                
                

            print("\n")
                
    
    def debug(self):
        pass
            
            
    def calc_fitnes(self, individu, heuristic):
        wh = heuristic*individu
        return np.sum(wh[:self.num_of_gen-1]) - wh[-1]
    
    def self_scan(self):
        data = {}
        for idx, lubang in enumerate(self.self_state):
            h1 = self.h1(idx, lubang)
            h2 = self.h2(idx, lubang)
            h3 = self.h3(idx, lubang)
            h4 = self.h4(idx, lubang)
            h5 = self.h5(idx, lubang)
            
            data[idx] = np.array([h1,h2,h3,h4,h5])
        return data
    
    def candidate(self):
        pass
        
    
    def h1(self, idx_lubang, isi_lubang):
        '''
        Timbun sebanyak mungkin rock dalam satu lubang. 
        Karena pada akhir permainan, rock pada lubang akan dipindahkan semuanya pada lumbung. Lubang kanan semakin aman.
        '''
        # [0,1,2,3,4,5,6] [lumbung]
        if len(self.self_state)-(idx_lubang+isi_lubang) == 0:
            return 1.0
        return 0.0
    
    def h2(self, idx_lubang, isi_lubang):
        '''
        Pertahankan agar rock pada sisi kita sebanyak mungkin. (versi umum H1)
        '''
        if isi_lubang == 0:
            return 0.0
        
        total_batu_before = np.sum(self.self_state)
        
        # simulasikan perpindahan batu pada lubang
        cp_state = self.self_state[:]
        
            
        # pecah list menjadi kiri dan kanan
        kiri = cp_state[:idx_lubang]
        tengah = [0]
        kanan = cp_state[idx_lubang+1:]
        
        jml_batu = isi_lubang
        new_kanan = []
        for i in kanan:
            
            if jml_batu == 0:
                break
            else:
                new_kanan.append(i+1)
            
            jml_batu -= 1
        
        new_state = kiri+tengah+kanan

        
        # jumlahkan setiap batu untuk kondisi terbaru
        total_batu_after = np.sum(new_state)
        
        return total_batu_after/total_batu_before
    
    def h3(self, idx_lubang, isi_lubang):
        '''
        Mempertahankan gerakan memindah rock sebanyak mungkin
        '''
        # [0,1,2,3,4,5,6] [lumbung]
        if len(self.self_state) == (idx_lubang+isi_lubang):
            return 1.0
        return 0.0
    
    def h4(self, idx_lubang, isi_lubang):
        '''
        Memaksimalkan jumlah rock pada lumbung. Dengan memilih langkah mencuri
        '''
        target = idx_lubang + isi_lubang
        
        if isi_lubang == 0:
            return 0.0
        elif 0 not in self.self_state:
            return 0.0
        elif target >= len(self.self_state):
            # out of range
            return 0.0
        elif self.self_state[target] == 0:
            # jika target nilainya 0
            # curi
            jml_curi = self.opponent_state[target]
            return 1-(1/jml_curi)
        else:
            return 0.0
#         if isi_lubang == 0:
#             return 0.0
        
#         if 0 not in self.self_state:
#             # jika ga ada 0 dalam lubang kita, berarti ga ada bisa yang dicuri
#             return 0.0
        
#         # jumlah perpindahan batu
#         jml_pindah = idx_lubang+isi_lubang
#         # print('jumlah pindah ', jml_pindah)
        
#         # jika kelebihan langkah batu
#         if len(self.self_state) < jml_pindah:
#             # out of range
#             return 0.0
        
#         if self.self_state[jml_pindah] == 0 and jml_pindah <= len(self.self_state):
#             # lakukan pencurian
#             jml_curi = self.opponent_state[jml_pindah]
#             return 1-(1/jml_curi)
#         else:
#             # ga ada yang bisa dicuri
#             return 0.0

        
    def h4_new(self, idx_lubang, isi_lubang):
        
        target = idx_lubang + isi_lubang
        
        if isi_lubang == 0:
            return 0.0
        elif 0 not in self.self_state:
            return 0.0
        elif target > len(self.self_state):
            # out of range
            return 0.0
        elif self.self_state[target] == 0:
            # jika target nilainya 0
            # curi
            jml_curi = self.opponent_state[jml_pindah]
            return 1-(1/jml_curi)
        else:
            return 0.0
        
    
    def h5(self, idx_lubang, isi_lubang):
        '''
        h5 pada excel dihapus diganti dengan h6 diexcel
        yaitu strategi bertahan
        Menjaga score musuh seminimal mungkin. (heuristik dengan mempertimbangkan 2 gerakan musuh kedepan.
        '''
        # [0,1,2,3,4,5,6] [lumbung]
        if isi_lubang == 0:
            return 0.0
        
        jml_batu_ke_musuh = abs(len(self.self_state) - (idx_lubang+isi_lubang))
        if jml_batu_ke_musuh > 0: # berarti ada batu yang masuk ke lubang musuh
            jml_batu_ke_musuh = len(self.self_state) if jml_batu_ke_musuh > len(self.self_state) else jml_batu_ke_musuh
            return jml_batu_ke_musuh/len(self.self_state)
        else:
            return 0.0
    
    def turnamen(individu1, individu2):
        pass
    
    def selection(self, population, verbose=False):
        '''
        menggunakan metode turnamen dari individu yang berada dalam kandidat
        '''
        scan = self.self_scan()
        
        self.kandidat = []
        
        self.rata_fitness = [0,0,0,0,0,0,0,0,0,0]
        for lubang, heuristic in scan.items():
            if verbose:
                print(f"===========================lubang {lubang}======================================")
                print("\n")
                temp = []
                for idx, individu in enumerate(population):
                    data_individu = {
                        'lubang': lubang,
                        'heuristic': heuristic,
                        'individu': {
                            'individuke':idx,
                            'gen':individu
                        },
                        'fitness': self.calc_fitnes(individu, heuristic)
                    }
                    
                    
                    temp.append(data_individu)
                    self.rata_fitness[idx] += data_individu['fitness']
                    print(data_individu)
                    print("\n")
                

                best_individu_each_lubang = sorted(temp, key=lambda p : p.get('fitness'), reverse=True)
                print(f'best individu di lubang {lubang} adalah {best_individu_each_lubang[0]}')
                # append to kandidat
                self.kandidat.append(best_individu_each_lubang[0])
                print(f"================================================================================")
                print("\n")
                
                
                # masukan individu terbaik dari setiap lubang kedalam list kandidat
                
            else:
                pass
            
    def turnamen(self, pop):
        kandidat= []
        result = self.rata_fitness
        while ((len(kandidat)<4)):
            x = random.randint(0,9)
            if(x not in kandidat):
                kandidat.append(x)
            else:
                pass
#         print("daftar kandidat yang akan di turnamentkan ", kandidat)
#         #kandidat[0] vs kandidat[1] , kandidat[2] vs kandidat[3]
#         print(pop[kandidat[0]]," Dengan Fitness ",result[kandidat[0]])
#         print(pop[kandidat[1]]," Dengan Fitness ",result[kandidat[1]])
#         print("=============================================================")
#         print(pop[kandidat[2]]," Dengan Fitness ",result[kandidat[2]])
#         print(pop[kandidat[3]]," Dengan Fitness ",result[kandidat[3]])
#         print("=======================TOURNAMENT DIMULAI=============================")
        if(result[kandidat[0]] > result[kandidat[1]]):
            x = pop[kandidat[0]]
        else:
            x = pop[kandidat[1]]

        if(result[kandidat[2]] > result[kandidat[3]]):
            y = pop[kandidat[2]]
        else:
            y = pop[kandidat[3]]
#         print("========================================================================")
#         print("x = ",x)
#         print("y = ",y)
        return x, y
    
    def crossover(self, individu1, individu2):
        '''
        crossover dengan single point mutation
        
        return new offspring
        '''
        prandom = np.random.uniform(0,1)
        
        if prandom < self.pc:
            # lakukan crossover
            point = random.randint(0,3)
        
            papa = list(individu1[:point])
            
            
            mama = list(individu2[point:])
    
            
            offspring = papa+mama 
            
            
#             return offspring
            return np.array(offspring)        
        else:
            '''
            return kan random mama atau papa
            '''
            return np.random.choice(list[individu1, individu2])
    
    def mutation(self, individu, num_of_gen):
        '''
        mutasi
        lakukan mutasi hanya untuk 2 gen dalam 1 individu jika propbabiliti individu lebih kecil dari pm
        '''
        pidv = np.random.uniform(0,1)
        print(pidv)
        if pidv < self.pm:
            
            # lakukan pemilihan random gen
            gen_bermutasi = np.random.choice([i for i in range(self.num_of_gen)], num_of_gen)
            print(gen_bermutasi)
            
            for i in gen_bermutasi:
                
                individu[i] = np.random.uniform(0,100)
            
            return individu
        
        else:
            return individu
        
    def elitism(self):
        
        pass

## RUN

In [265]:
'''
setting param
'''


SELF_STATE = [1,0,2,3,0,5,0]
OPPONENT_STATE = [6,7,8,9,2,4,12]
NUMBER_OF_GEN = 5
NUMBER_OF_POPULATION = 10
NUMBER_OF_GENERATION = 10
PC = 0.8
PM = 0.5


ga = MancalaGA(SELF_STATE, OPPONENT_STATE, NUMBER_OF_GEN, NUMBER_OF_POPULATION, NUMBER_OF_GENERATION, PC, PM)

In [266]:
ga.self_scan()

{0: array([0.        , 0.90909091, 0.        , 0.85714286, 0.85714286]),
 1: array([0., 0., 0., 0., 0.]),
 2: array([0.        , 0.81818182, 0.        , 0.5       , 0.42857143]),
 3: array([0.        , 0.72727273, 0.        , 0.91666667, 0.14285714]),
 4: array([0., 0., 0., 0., 0.]),
 5: array([0.        , 0.54545455, 0.        , 0.        , 0.42857143]),
 6: array([0., 0., 0., 0., 0.])}

In [267]:
pop = ga.population()
pop

array([[ 0.89506987, 74.81062629, 68.79910214, 80.98952159, 25.74316325],
       [15.13724161, 26.9657993 , 96.89010867, 47.89640948, 46.14690597],
       [82.16760857, 28.47625156, 41.00896732, 91.49412492, 75.56947824],
       [67.21198905, 12.28902921,  0.50769361, 13.84089815, 79.50416996],
       [ 4.51772041, 81.40236776, 45.96535842, 28.10049256, 80.94661399],
       [52.16512525, 52.53009992, 73.25825426, 48.72573581, 43.29667834],
       [50.49121414, 44.51788362, 59.74476197, 30.02102315, 36.88434144],
       [76.90589334, 43.37274427, 65.95237771, 41.16772883, 46.9171289 ],
       [66.75230952, 58.44061278, 65.51518905,  1.3890407 , 33.35681077],
       [55.89260443, 44.06839909, 26.86742073, 56.21588982, 88.19508894]])

In [268]:
ga.selection(pop, verbose=True)

===========================lubang 0======================================


{'lubang': 0, 'heuristic': array([0.        , 0.90909091, 0.        , 0.85714286, 0.85714286]), 'individu': {'individuke': 0, 'gen': array([ 0.89506987, 74.81062629, 68.79910214, 80.98952159, 25.74316325])}, 'fitness': 115.36368169384674}


{'lubang': 0, 'heuristic': array([0.        , 0.90909091, 0.        , 0.85714286, 0.85714286]), 'individu': {'individuke': 1, 'gen': array([15.13724161, 26.9657993 , 96.89010867, 47.89640948, 46.14690597])}, 'fitness': 26.01393744501042}


{'lubang': 0, 'heuristic': array([0.        , 0.90909091, 0.        , 0.85714286, 0.85714286]), 'individu': {'individuke': 2, 'gen': array([82.16760857, 28.47625156, 41.00896732, 91.49412492, 75.56947824])}, 'fitness': 39.53719857098332}


{'lubang': 0, 'heuristic': array([0.        , 0.90909091, 0.        , 0.85714286, 0.85714286]), 'individu': {'individuke': 3, 'gen': array([67.21198905, 12.28902921,  0.50769361, 13.84089815, 79.50416996

In [269]:
ga.kandidat

[{'lubang': 0,
  'heuristic': array([0.        , 0.90909091, 0.        , 0.85714286, 0.85714286]),
  'individu': {'individuke': 0,
   'gen': array([ 0.89506987, 74.81062629, 68.79910214, 80.98952159, 25.74316325])},
  'fitness': 115.36368169384674},
 {'lubang': 1,
  'heuristic': array([0., 0., 0., 0., 0.]),
  'individu': {'individuke': 0,
   'gen': array([ 0.89506987, 74.81062629, 68.79910214, 80.98952159, 25.74316325])},
  'fitness': 0.0},
 {'lubang': 2,
  'heuristic': array([0.        , 0.81818182, 0.        , 0.5       , 0.42857143]),
  'individu': {'individuke': 0,
   'gen': array([ 0.89506987, 74.81062629, 68.79910214, 80.98952159, 25.74316325])},
  'fitness': 90.6706707794841},
 {'lubang': 3,
  'heuristic': array([0.        , 0.72727273, 0.        , 0.91666667, 0.14285714]),
  'individu': {'individuke': 0,
   'gen': array([ 0.89506987, 74.81062629, 68.79910214, 80.98952159, 25.74316325])},
  'fitness': 124.97052824844849},
 {'lubang': 4,
  'heuristic': array([0., 0., 0., 0., 0.])

In [270]:
ga.rata_fitness

[360.777892628699,
 104.10331315305423,
 153.12565056473494,
 -79.31194769828616,
 157.7728449059139,
 187.975224982183,
 133.31624796154057,
 136.59399587110565,
 116.53188943928039,
 96.23854348907784]

In [280]:
turnamen = ga.turnamen(pop)
turnamen

(array([52.16512525, 52.53009992, 73.25825426, 48.72573581, 43.29667834]),
 array([ 0.89506987, 74.81062629, 68.79910214, 80.98952159, 25.74316325]))

In [281]:
c = ga.crossover(turnamen[0], turnamen[1])
c

array([ 0.89506987, 74.81062629, 68.79910214, 80.98952159, 25.74316325])

In [284]:
ga.mutation(c,2)

array([76.32040398, 51.22122298, 68.79910214, 11.52654762, 99.94817143])